In [0]:
import tensorflow as tf
print("GPU Available: ", tf.config.experimental.list_physical_devices('GPU'))
print("Tensorflow Version: ", tf.__version__)

GPU Available:  []
Tensorflow Version:  2.1.0


In [0]:
!pwd

/data/cmpe297-05-sp20/siddartha/TensorFlow/hw2


In [0]:
#!pip install pycocotools

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [0]:
#!pip install pytube3 #https://pypi.org/project/pytube3/

In [0]:
import pytube
video_url = 'https://www.youtube.com/watch?v=gY8E8ILFkzo'
#video_url = 'https://www.youtube.com/watch?v=NcaGFp76BTY'
youtube = pytube.YouTube(video_url)
video = youtube.streams.first()

In [0]:
!pwd
#!mkdir videos

/data/cmpe297-05-sp20/siddartha/TensorFlow/hw2
mkdir: cannot create directory ‘videos’: File exists


In [0]:
#video.download('videos/')

'/data/cmpe297-05-sp20/siddartha/TensorFlow/hw2/videos/Road Rage captured on Tesla Model 3 Dash Cam.mp4'

In [0]:
#!mv ./videos/*.mp4 testvideo1.mp4

In [0]:
video_path  = "./challenge_video.mp4"#testvideo1.mp4"

In [0]:
PATH_TO_FROZEN_GRAPH = './trained-inference-graphs/output_inference_graph_v1.pb/frozen_inference_graph.pb'

In [0]:
PATH_TO_LABELS = './annotations/kitti_map.pbtxt'

In [0]:
import os
import pathlib
#!pip install opencv-python
import cv2
import time
#!apt install python3-opencv
print(cv2.__version__)

4.2.0


In [0]:
#!pip install --upgrade imutils
from imutils.video import FPS
import imutils

In [0]:
model_dir = './trained-inference-graphs/output_inference_graph_v1.pb/saved_model'
model = tf.saved_model.load(str(model_dir))
model = model.signatures['serving_default']
#model2 = model2.signatures['serving_default']
print(model.inputs)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [0]:
model.output_shapes

{'raw_detection_scores': TensorShape([None, 300, 10]),
 'detection_multiclass_scores': TensorShape([None, 300, 10]),
 'detection_classes': TensorShape([None, 300]),
 'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 300, 4]),
 'raw_detection_boxes': TensorShape([None, 300, 4]),
 'detection_features': TensorShape([None, None, None, None, None]),
 'detection_scores': TensorShape([None, 300])}

In [0]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './annotations/kitti_map.pbtxt'
#PATH_TO_LABELS2 = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
#category_index2 = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS2, use_display_name=True)

In [0]:
#vim 

In [0]:
#!pip install --upgrade imutils

In [0]:
from imutils.video import FPS
import imutils

In [0]:
detectionmodelnew=model

cap = cv2.VideoCapture(video_path)
print(cap.get(cv2.CAP_PROP_FPS))
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
print(frame_width)
print(frame_height)
fps = FPS().start()

# Define the codec and create VideoWriter object.
#out = cv2.VideoWriter('./output_video2.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
out = cv2.VideoWriter('./challenge_lane.mp4',cv2.VideoWriter_fourcc('M','P','4','V'), 30, (frame_width,frame_height))

i=0

#frame 
currentframe = 0
while (cap.isOpened() and currentframe<2000):
  ret, image_np = cap.read()
  if len((np.array(image_np)).shape) == 0:
    break

  #image_np_expanded = np.expand_dims(image_np, axis=0)

  image = np.asarray(image_np)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]
  # Run inference
  #output_dict = run_inference_for_single_image(detectionmodelnew, image_np)
  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
  
  #print(output_dict['detection_classes'])
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  #display(Image.fromarray(image_np))
  #cv2_imshow('object detection', cv2.resize(image_np, (800,600)))
  #cv2_imshow(cv2.resize(image_np, (640,360)))

  name = './Test_data/frame' + str(currentframe) + '.jpg'
  print ('Creating\...' + name) 
  cv2.imwrite(name, image_np) 
  # increasing counter so that it will 
            # show how many frames are created 
  currentframe += 1
            
  fps.update()
  out.write(image_np)

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
cap.release()
out.release()
cv2.destroyAllWindows()

25.0
1280
720
Creating\..../Test_data/frame0.jpg
Creating\..../Test_data/frame1.jpg
Creating\..../Test_data/frame2.jpg
Creating\..../Test_data/frame3.jpg
Creating\..../Test_data/frame4.jpg
Creating\..../Test_data/frame5.jpg
Creating\..../Test_data/frame6.jpg
Creating\..../Test_data/frame7.jpg
Creating\..../Test_data/frame8.jpg
Creating\..../Test_data/frame9.jpg
Creating\..../Test_data/frame10.jpg
Creating\..../Test_data/frame11.jpg
Creating\..../Test_data/frame12.jpg
Creating\..../Test_data/frame13.jpg
Creating\..../Test_data/frame14.jpg
Creating\..../Test_data/frame15.jpg
Creating\..../Test_data/frame16.jpg
Creating\..../Test_data/frame17.jpg
Creating\..../Test_data/frame18.jpg
Creating\..../Test_data/frame19.jpg
Creating\..../Test_data/frame20.jpg
Creating\..../Test_data/frame21.jpg
Creating\..../Test_data/frame22.jpg
Creating\..../Test_data/frame23.jpg
Creating\..../Test_data/frame24.jpg
Creating\..../Test_data/frame25.jpg
Creating\..../Test_data/frame26.jpg
Creating\..../Test_data/

Creating\..../Test_data/frame225.jpg
Creating\..../Test_data/frame226.jpg
Creating\..../Test_data/frame227.jpg
Creating\..../Test_data/frame228.jpg
Creating\..../Test_data/frame229.jpg
Creating\..../Test_data/frame230.jpg
Creating\..../Test_data/frame231.jpg
Creating\..../Test_data/frame232.jpg
Creating\..../Test_data/frame233.jpg
Creating\..../Test_data/frame234.jpg
Creating\..../Test_data/frame235.jpg
Creating\..../Test_data/frame236.jpg
Creating\..../Test_data/frame237.jpg
Creating\..../Test_data/frame238.jpg
Creating\..../Test_data/frame239.jpg
Creating\..../Test_data/frame240.jpg
Creating\..../Test_data/frame241.jpg
Creating\..../Test_data/frame242.jpg
Creating\..../Test_data/frame243.jpg
Creating\..../Test_data/frame244.jpg
Creating\..../Test_data/frame245.jpg
Creating\..../Test_data/frame246.jpg
Creating\..../Test_data/frame247.jpg
Creating\..../Test_data/frame248.jpg
Creating\..../Test_data/frame249.jpg
Creating\..../Test_data/frame250.jpg
Creating\..../Test_data/frame251.jpg
C

Creating\..../Test_data/frame447.jpg
Creating\..../Test_data/frame448.jpg
Creating\..../Test_data/frame449.jpg
Creating\..../Test_data/frame450.jpg
Creating\..../Test_data/frame451.jpg
Creating\..../Test_data/frame452.jpg
Creating\..../Test_data/frame453.jpg
Creating\..../Test_data/frame454.jpg
Creating\..../Test_data/frame455.jpg
Creating\..../Test_data/frame456.jpg
Creating\..../Test_data/frame457.jpg
Creating\..../Test_data/frame458.jpg
Creating\..../Test_data/frame459.jpg
Creating\..../Test_data/frame460.jpg
Creating\..../Test_data/frame461.jpg
Creating\..../Test_data/frame462.jpg
Creating\..../Test_data/frame463.jpg
Creating\..../Test_data/frame464.jpg
Creating\..../Test_data/frame465.jpg
Creating\..../Test_data/frame466.jpg
Creating\..../Test_data/frame467.jpg
Creating\..../Test_data/frame468.jpg
Creating\..../Test_data/frame469.jpg
Creating\..../Test_data/frame470.jpg
Creating\..../Test_data/frame471.jpg
Creating\..../Test_data/frame472.jpg
Creating\..../Test_data/frame473.jpg
C

Creating\..../Test_data/frame669.jpg
Creating\..../Test_data/frame670.jpg
Creating\..../Test_data/frame671.jpg
Creating\..../Test_data/frame672.jpg
Creating\..../Test_data/frame673.jpg
Creating\..../Test_data/frame674.jpg
Creating\..../Test_data/frame675.jpg
Creating\..../Test_data/frame676.jpg
Creating\..../Test_data/frame677.jpg
Creating\..../Test_data/frame678.jpg
Creating\..../Test_data/frame679.jpg
Creating\..../Test_data/frame680.jpg
Creating\..../Test_data/frame681.jpg
Creating\..../Test_data/frame682.jpg
Creating\..../Test_data/frame683.jpg
Creating\..../Test_data/frame684.jpg
Creating\..../Test_data/frame685.jpg
Creating\..../Test_data/frame686.jpg
Creating\..../Test_data/frame687.jpg
Creating\..../Test_data/frame688.jpg
Creating\..../Test_data/frame689.jpg
Creating\..../Test_data/frame690.jpg
Creating\..../Test_data/frame691.jpg
Creating\..../Test_data/frame692.jpg
Creating\..../Test_data/frame693.jpg
Creating\..../Test_data/frame694.jpg
Creating\..../Test_data/frame695.jpg
C

Creating\..../Test_data/frame891.jpg
Creating\..../Test_data/frame892.jpg
Creating\..../Test_data/frame893.jpg
Creating\..../Test_data/frame894.jpg
Creating\..../Test_data/frame895.jpg
Creating\..../Test_data/frame896.jpg
Creating\..../Test_data/frame897.jpg
Creating\..../Test_data/frame898.jpg
Creating\..../Test_data/frame899.jpg
Creating\..../Test_data/frame900.jpg
Creating\..../Test_data/frame901.jpg
Creating\..../Test_data/frame902.jpg
Creating\..../Test_data/frame903.jpg
Creating\..../Test_data/frame904.jpg
Creating\..../Test_data/frame905.jpg
Creating\..../Test_data/frame906.jpg
Creating\..../Test_data/frame907.jpg
Creating\..../Test_data/frame908.jpg
Creating\..../Test_data/frame909.jpg
Creating\..../Test_data/frame910.jpg
Creating\..../Test_data/frame911.jpg
Creating\..../Test_data/frame912.jpg
Creating\..../Test_data/frame913.jpg
Creating\..../Test_data/frame914.jpg
Creating\..../Test_data/frame915.jpg
Creating\..../Test_data/frame916.jpg
Creating\..../Test_data/frame917.jpg
C

Creating\..../Test_data/frame1110.jpg
Creating\..../Test_data/frame1111.jpg
Creating\..../Test_data/frame1112.jpg
Creating\..../Test_data/frame1113.jpg
Creating\..../Test_data/frame1114.jpg
Creating\..../Test_data/frame1115.jpg
Creating\..../Test_data/frame1116.jpg
Creating\..../Test_data/frame1117.jpg
Creating\..../Test_data/frame1118.jpg
Creating\..../Test_data/frame1119.jpg
Creating\..../Test_data/frame1120.jpg
Creating\..../Test_data/frame1121.jpg
Creating\..../Test_data/frame1122.jpg
Creating\..../Test_data/frame1123.jpg
Creating\..../Test_data/frame1124.jpg
Creating\..../Test_data/frame1125.jpg
Creating\..../Test_data/frame1126.jpg
Creating\..../Test_data/frame1127.jpg
Creating\..../Test_data/frame1128.jpg
Creating\..../Test_data/frame1129.jpg
Creating\..../Test_data/frame1130.jpg
Creating\..../Test_data/frame1131.jpg
Creating\..../Test_data/frame1132.jpg
Creating\..../Test_data/frame1133.jpg
Creating\..../Test_data/frame1134.jpg
Creating\..../Test_data/frame1135.jpg
Creating\...

In [0]:
# output_dict['detection_classes']

In [0]:
# import tensorflow as tf

# for example in tf.compat.v1.python_io.tf_record_iterator("annotations/valid.tfrecord2"):
#     result = tf.train.SequenceExample.FromString(example)
#     break

#print(result)

In [0]:
# for feature in result:
#     print feature